In [1]:

import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, Input,Layer
from scipy.fftpack import dct
# Configurazione per la GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Limita la memoria GPU allocata (opzionale)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Permette allocazione dinamica della memoria
        print(f"{len(gpus)} GPU(s) disponibile/i.")
    except RuntimeError as e:
        print(e)
else:
    print("Nessuna GPU disponibile, verrà usata la CPU.")

# Carica il dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

def apply_dct(data):
    # Esegui la DCT bidimensionale sull'immagine
    # La `type=2` rappresenta la DCT-II, che è la più comune
    return dct(dct(data, axis=0, norm='ortho'), axis=1, norm='ortho')

X_train_dct = np.array([apply_dct(img) for img in X_train])  # Applica la DCT alle immagini di training
X_test_dct = np.array([apply_dct(img) for img in X_test])  # Applica la DCT alle immagini di test


print("Forma di X_train_dct:", X_train_dct.shape)
print("Forma di X_test_dct:", X_test_dct.shape)

2024-12-19 13:20:38.596859: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-19 13:20:38.613670: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-19 13:20:38.618318: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-19 13:20:38.628954: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1734610840.683300   104

1 GPU(s) disponibile/i.
Forma di X_train_dct: (60000, 28, 28)
Forma di X_test_dct: (10000, 28, 28)


In [2]:
filtri = np.random.rand(32, 3, 3)

filtri_dct = np.array([apply_dct(img) for img in filtri])  # Applica la DCT alle immagini di training


# Stampa i filtri generati
for i, filtro in enumerate(filtri_dct):
    print(f"Filtro {i+1}:\n{filtro}\n")

Filtro 1:
[[ 2.01490484  0.02273739  0.20807512]
 [ 0.19517652  0.03093649 -0.28780002]
 [ 0.09046218 -0.66583017  0.2467256 ]]

Filtro 2:
[[ 1.80026887 -0.39697953 -0.4168278 ]
 [ 0.08682446  0.21602908  0.34059942]
 [-0.18385539  0.19957312 -0.20151331]]

Filtro 3:
[[ 1.22126617  0.0052015  -0.52742093]
 [ 0.33784643  0.31413849  0.04425967]
 [-0.09331925  0.25581318  0.4074057 ]]

Filtro 4:
[[ 1.74423621  0.08263877  0.03724123]
 [ 0.26600278  0.27898351 -0.49202909]
 [ 0.21418271  0.25432162  0.45278172]]

Filtro 5:
[[ 1.746036   -0.06963857  0.11756317]
 [-0.29673794 -0.22064443 -0.29941863]
 [-0.21229563 -0.10733693 -0.07005113]]

Filtro 6:
[[ 1.41770229  0.10683736 -0.28181234]
 [-0.14704586  0.53902321  0.40452269]
 [ 0.14531329  0.28576959  0.2211669 ]]

Filtro 7:
[[ 1.72189739 -0.38257106 -0.4899413 ]
 [-0.35985546 -0.0539674  -0.0224057 ]
 [ 0.40644888 -0.14266853  0.32575866]]

Filtro 8:
[[ 1.82956308 -0.18236075  0.27479529]
 [ 0.14967565  0.66625764  0.09246519]
 [ 0.0629

In [3]:
#rimuovi tutte le colonne apparte prime 3 in tutte le immagini di training e test
X_train_dct = X_train_dct[:, :, :3]
X_test_dct = X_test_dct[:, :, :3]



#printa le shape

print("Forma di X_train_dct:", X_train_dct.shape)
print("Forma di X_test_dct:", X_test_dct.shape)


Forma di X_train_dct: (60000, 28, 3)
Forma di X_test_dct: (10000, 28, 3)


In [4]:
class IDCTLayer(Layer):
    def __init__(self, **kwargs):
        super(IDCTLayer, self).__init__(**kwargs)

    def call(self, inputs):
        # IDCT lungo l'asse delle righe e poi delle colonne
        def idct_2d(tensor):
            # Applica la IDCT lungo le righe (asse=-1)
            x = tf.signal.idct(tensor, type=2, norm='ortho', axis=-1)
            # Trasponi per applicare la IDCT lungo le colonne
            x = tf.transpose(x, perm=[0, 1, 3, 2])  # Scambia -2 (righe) con -1 (colonne)
            x = tf.signal.idct(x, type=2, norm='ortho', axis=-1)
            # Ritorna alla disposizione originale
            x = tf.transpose(x, perm=[0, 1, 3, 2])  # Ritorna alla forma originale
            return x

        # Applica la IDCT batch-wise e canale per canale
        return idct_2d(inputs)

In [ ]:
model = Sequential([
    Input(shape=(28, 3)),  
    Flatten(),             
    Dense(228, activation='relu'),
    Dense(576, activation='relu'),
    Dense(1152, activation='relu'),
    Reshape((3, 3, 128)),
    IDCTLayer(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

I0000 00:00:1734610842.567370   10405 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734610842.567706   10405 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734610842.567921   10405 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734610842.686882   10405 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [6]:
# Compila il modello
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Addestra il modello
history = model.fit(X_train_dct, y_train, epochs=30, batch_size=64, validation_split=0.2)

# Valuta il modello
test_loss, test_acc = model.evaluate(X_test_dct, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.2f}")


Epoch 1/30


I0000 00:00:1734610844.391328   10460 service.cc:146] XLA service 0x7c9e10002840 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734610844.391350   10460 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1060, Compute Capability 6.1
2024-12-19 13:20:44.419633: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-19 13:20:44.580085: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101


 63/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4206 - loss: 36.8043

I0000 00:00:1734610846.111395   10460 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


750/750 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6864 - loss: 6.7485 - val_accuracy: 0.7990 - val_loss: 0.5650
Epoch 2/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8192 - loss: 0.5059 - val_accuracy: 0.8278 - val_loss: 0.4835
Epoch 3/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8360 - loss: 0.4462 - val_accuracy: 0.8346 - val_loss: 0.4635
Epoch 4/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8493 - loss: 0.4105 - val_accuracy: 0.8317 - val_loss: 0.4646
Epoch 5/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8531 - loss: 0.3902 - val_accuracy: 0.8393 - val_loss: 0.4531
Epoch 6/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8648 - loss: 0.3680 - val_accuracy: 0.8404 - val_loss: 0.4576
Epoch 7/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8661 - loss: 0.3559 - val_accuracy: 0.8475 - val_loss: 0.4404
Epoch 8/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8726 - loss: 0.3434 - val_accuracy: 0.8487 - val_

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape

# Configurazione per la GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Limita la memoria GPU allocata (opzionale)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Permette allocazione dinamica della memoria
        print(f"{len(gpus)} GPU(s) disponibile/i.")
    except RuntimeError as e:
        print(e)
else:
    print("Nessuna GPU disponibile, verrà usata la CPU.")

# Carica il dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()



# Crea il modello
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    Conv2D(64, (3, 3), activation='relu'),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])


# Compila il modello
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Addestra il modello
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Valuta il modello
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.2f}")


1 GPU(s) disponibile/i.
Epoch 1/10
319/750 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7094 - loss: 10.2375

KeyboardInterrupt: 